In [1]:
from pyvis import network as net
import networkx as nx
from scripts.address import case_2_address, case_2_wallet
from src.blockvision import get_transactions_by_wallet
import asyncio
from src.safe import get_owners_by_wallet
import pickle
from src.contracts import FTX_Exchange_2, OKEX, OKEX2, BINANCE_14, OP_TELEPORTER

# Related Safe Addresses

In [2]:
print('\n'.join(case_2_wallet))

0x744f29411464a4313584938879f72e97dafc9ade
0x72068eee5db83a88a2970d856d005ed0ef7e4f96
0xbbc1c6b32829f9289bd4c8d208ae80a18294258f
0x612da9b442dadeae1539127c3895469a5111306b
0xce7c6f34fda1fbdbbb9f14dd9a411504e2a0e74e
0x4c9deabb4082a486fb7ac06ed755e79a84618a65
0x452298a9a026bec7e60eee9c9f5f10930f4ff9a0
0x06f7a40cfbe1b13755fd0ab9ce35494e52dc0140
0x56ac8761cfce4ecde304c77e0e8569fdfc81ca7d
0xff0e43745d241e12ce3fce5dbf814a74e332c481


## EOA address

In [3]:
print('\n'.join(case_2_address))

0x548e1b7bb250e4c9eea46fc2874c43d81a914c54
0xd85bf6feeee6270f2ca53aa7dee7590668431133
0x94feb079aac8ab01ea5460ba041a2c0be8629d33
0xd60e645c391ed4374829d7ec6439b7e1c7a5438c
0x21671a68b84fc1d9b17be7123990c49c223ea70e
0x7d64c23101effd90abd56c8734932cc78c8c5c65
0xdd13d45df2b081867b1210d954444db86a699999
0x0dbe81313d9813e420e07d52e03923b431b39999
0x5ee1aa6537cadd807ecd5ab8ee8f1811b0888888
0x8f5dad7f4db78df43a082ebe6a9b8311a8888888
0xf528803f1a9aa49d7bef51c08f95674b52b88888
0x779ee541b6af2d5da83c4479ddbd9c7b63668888
0xf11dd56e87d49c4edb2710054f26048bf1d17777
0xf9696752b2122342f4600526f91a86c971566666
0x85c0625eaca383da1855591f2f34c19070b56666
0x2db60d2cb464c9e1a8d0fa5b782574cfcf345555
0x316a67627e54b231afe1bb283645dcd8b48f4444
0x61349fffa36cdfbaddb71fbfc6434516807b3333
0xf7c57bf44dd46d0295dc724a23b014976c9a2222
0x158565947b23f195167c353b9d8916e84cd01111
0x90e1e8584340bfd3b3c19b676273361fc1565d0e
0x2a25bd8a1a4f2c255e1f64756207c95399131265
0xba62698e5b2d29ea36f40c67be1a5936f532dd18
0x5b4980c8f

## Reasoning

I find part of address by filter small nonce address from safe wallet's owners.
These address have many transfer ETH each other, so I find all address control by one people.
He create 10 Safe wallets, add 3-6 owners each.
Except for transfers to the exchange and one to OP-bridge, **none of transfers went outside**. It can be determined that all wallets are controlled by one person.

In [4]:
pickle_path_0 = "pickles/13_address_txs.pickle"
pickle_path_1 = "pickles/13_wallet_to_owners.pickle"

query = False # set true if download txs

g = net.Network(notebook=True, directed =True)
combine = [FTX_Exchange_2, OKEX, OKEX2, BINANCE_14, OP_TELEPORTER]+case_2_address+case_2_wallet
for index, address in enumerate(combine):
    if address == OP_TELEPORTER:
        g.add_node(index, label=f"OP bridge", color="#ff0000")
    elif index < 4:
        g.add_node(index, label=f"Exchange", shape='box', color="#FFD700")
    elif index < 4 + len(case_2_address):
        g.add_node(index, label=f"Wallet: {address[:5]}...{address[-4:]}", shape='dot')        
    else:
        g.add_node(index, label=f"Safe: {address[:5]}...{address[-4:]}", shape='dot', color='#00FA9A')

if query:
    address_to_transactions = {}
    for address in case_2_address:
        print(f"downloading {address}")
        result = await get_transactions_by_wallet(address)
        address_to_transactions[address] = result

    with open(pickle_path_0, "wb") as f:
        pickle.dump(address_to_transactions, f)

    wallet_to_owners = get_owners_by_wallet(case_2_wallet)
    with open(pickle_path_1, "wb") as f:
        print(wallet_to_owners)
        pickle.dump(wallet_to_owners, f)
else:
    # all owners in case_2_address
    with open(pickle_path_1, "rb") as f:
        wallet_to_owners = pickle.load(f)

    for wallet, owners in wallet_to_owners.items():
        is_all_signer_in_case = all([owner in case_2_address for owner in owners])
        print(
            f"safe {wallet}  have {len(owners)} signer, all signer in case_2_address: {is_all_signer_in_case}"
        )
        for owner in owners:
            from_index = combine.index(owner)
            to_index = combine.index(wallet)
            g.add_edge(from_index, to_index)

    # find transfer tx
    with open(pickle_path_0, "rb") as f:
        address_to_transactions = pickle.load(f)

    for address, txs in address_to_transactions.items():
        if txs["data"] is None:
            continue
        for tx in txs["data"]:
            if tx["inputData"] == "0x" and tx["from"] == address:
                to = tx["to"]
                from_index = combine.index(address)
                to_index = combine.index(to)
                g.add_edge(from_index, to_index, width=4)
                if to in case_2_address:
                    print(
                        f"address {address} (txs: {txs['total']}) transfer to case_2_address[{case_2_address.index(to)}]"
                    )
                elif to in case_2_wallet:
                    print(
                        f"address {address}(txs: {txs['total']}) transfer to case_2_wallet[{case_2_wallet.index(to)}]"
                    )
                elif to == FTX_Exchange_2:
                    print(
                        f"address {address}(txs: {txs['total']}) transfer to FTX_Exchange_2"
                    )
                elif to == OKEX:
                    print(
                        f"address {address}(txs: {txs['total']}) transfer to OKEX"
                    )
                elif to == OKEX2:
                    print(
                        f"address {address}(txs: {txs['total']}) transfer to OKEX2"
                    )
                elif to == BINANCE_14:
                    print(
                        f"address {address}(txs: {txs['total']}) transfer to BINANCE_14"
                    )
                elif to == OP_TELEPORTER:
                    print(
                        f"address {address}(txs: {txs['total']}) transfer to OP Bridge"
                    )  
                else:
                    print(
                        f"address {address}(txs: {txs['total']}) transfer to unknow {to}"
                    )
print('\n')
print("The thin line is owner to safe, the thick line is ETH transfer")                    
g.show("images/case2.html")

safe 0x744f29411464a4313584938879f72e97dafc9ade  have 4 signer, all signer in case_2_address: True
safe 0x72068eee5db83a88a2970d856d005ed0ef7e4f96  have 4 signer, all signer in case_2_address: True
safe 0xbbc1c6b32829f9289bd4c8d208ae80a18294258f  have 3 signer, all signer in case_2_address: True
safe 0x612da9b442dadeae1539127c3895469a5111306b  have 4 signer, all signer in case_2_address: True
safe 0xce7c6f34fda1fbdbbb9f14dd9a411504e2a0e74e  have 6 signer, all signer in case_2_address: True
safe 0x4c9deabb4082a486fb7ac06ed755e79a84618a65  have 6 signer, all signer in case_2_address: True
safe 0x452298a9a026bec7e60eee9c9f5f10930f4ff9a0  have 4 signer, all signer in case_2_address: True
safe 0x06f7a40cfbe1b13755fd0ab9ce35494e52dc0140  have 4 signer, all signer in case_2_address: True
safe 0x56ac8761cfce4ecde304c77e0e8569fdfc81ca7d  have 5 signer, all signer in case_2_address: True
safe 0xff0e43745d241e12ce3fce5dbf814a74e332c481  have 4 signer, all signer in case_2_address: True
address 0x